# Train a Hypergraph Message Passing Neural Network (HMPNN)

In this notebook, we will create and train a Hypergraph Message Passing Neural Network in the hypergraph domain. This method is introduced in the paper [Message Passing Neural Networks for
Hypergraphs](https://arxiv.org/abs/2203.16995) by Heydari et Livi 2022. We will use a benchmark dataset, Cora, a collection of 2708 academic papers and 5429 citation relations, to do the task of node classification. There are 7 category labels, namely `Case_Based`, `Genetic_Algorithms`, `Neural_Networks`, `Probabilistic_Methods`, `Reinforcement_Learning`, `Rule_Learning` and `Theory`.

Each document is initially represented as a binary vector of length 1433, standing for a unique subset of the words within the papers, in which a value of 1 means the presence of its corresponding word in the paper.

In [3]:
import torch
import torch_geometric.datasets as geom_datasets
from sklearn.metrics import accuracy_score
import numpy as np

from topomodelx.nn.hypergraph.hmpnn import HMPNN

torch.manual_seed(0)
np.random.seed(0)

If GPU's are available, we will make use of them. Otherwise, this will run on CPU.

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


# Pre-processing

Here we download the dataset. It contains initial representation of nodes, the adjacency information, category labels and train-val-test masks.

In [5]:
dataset = geom_datasets.Planetoid(root="/TopoModelX/data/cora", name="Cora")[0]

Below, we construct the incidence matrix ($B_1$) which is of shape $n_\text{nodes} \times n_\text{edges}$.

In [6]:
dataset["incidence_1"] = torch.sparse_coo_tensor(
    dataset["edge_index"], torch.ones(dataset["edge_index"].shape[1]), dtype=torch.long
)
dataset = dataset.to(device)

In [7]:
x_0s = dataset["x"]
y = dataset["y"]
incidence_1 = dataset["incidence_1"]

# Train the Neural Network

We then specify the hyperparameters and construct the model, the loss and optimizer.

In [8]:
class Network(torch.nn.Module):
    """ Network class that initializes the base model and readout layer.

    Base model parameters:
    ----------
    Reqired:
    in_channels : int
        Dimension of the input features.
    hidden_channels : int
        Dimension of the hidden features.

    Optitional:
    **kwargs : dict
        Additional arguments for the base model.
    
    Readout layer parameters:
    ----------
    out_channels : int
        Dimension of the output features.
    task_level : str
        Level of the task. Either "graph" or "node".        
    """
    def __init__(
            self, 
            in_channels, 
            hidden_channels,
            out_channels, 
            
            task_level="graph",
            **kwargs):
        super().__init__()
        
        # Define the model
        self.base_model = HMPNN(
            in_channels=in_channels,
            hidden_channels=hidden_channels,
            **kwargs
        )


        # Readout
        self.linear = torch.nn.Linear(hidden_channels, out_channels)
        self.out_pool = True if task_level == "graph" else False
        
    def forward(self, x_0, x_1, incidence_1):
        # Base model
        x_0, x_1 = self.base_model(x_0, x_1, incidence_1)

        # Pool over all nodes in the hypergraph 
        if self.out_pool is True:
            x = torch.max(x_0, dim=0)[0]
        else:
            x = x_0

        return self.linear(x)

Initialize the model

In [9]:
# Base model hyperparameters
in_channels = x_0s.shape[1]
hidden_channels = 128
n_layers=1

# Readout hyperparameters
out_channels = torch.unique(y).shape[0]
task_level = "graph" if out_channels==1 else "node"


model = Network(
    in_channels=in_channels,
    hidden_channels=hidden_channels,
    out_channels=out_channels, 
    n_layers=n_layers,
    task_level=task_level,
    ).to(device)

In [10]:

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()



train_mask = dataset["train_mask"]
val_mask = dataset["val_mask"]
test_mask = dataset["test_mask"]




Now it's time to train the model, looping over the network for a low amount of epochs. We keep training minimal for the purpose of rapid testing.

In [11]:
torch.manual_seed(0)
test_interval = 5
num_epochs=100


initial_x_1 = torch.zeros_like(x_0s)
for epoch in range(1, num_epochs + 1):
    model.train()
    optimizer.zero_grad()
    y_hat = model(x_0s, initial_x_1, incidence_1)
    loss = loss_fn(y_hat[train_mask], y[train_mask])
    loss.backward()
    optimizer.step()

    train_loss = loss.item()
    y_pred = y_hat.argmax(dim=-1)
    train_acc = accuracy_score(y[train_mask], y_pred[train_mask])
    
    if epoch % test_interval == 0:
        model.eval()
        
        y_hat = model(x_0s, initial_x_1, incidence_1)
        val_loss = loss_fn(y_hat[val_mask], y[val_mask]).item()
        y_pred = y_hat.argmax(dim=-1)
        val_acc = accuracy_score(y[val_mask], y_pred[val_mask])


        test_loss = loss_fn(y_hat[test_mask], y[test_mask]).item()
        y_pred = y_hat.argmax(dim=-1)
        test_acc = accuracy_score(y[test_mask], y_pred[test_mask])
        print(
            f"Epoch: {epoch + 1} train loss: {train_loss:.4f} train acc: {train_acc:.2f} "
            f" val loss: {val_loss:.4f} val acc: {val_acc:.2f}"
            f" test loss: {test_acc:.4f} val acc: {test_acc:.2f}"
        )

        


Epoch: 6 train loss: 1.2548 train acc: 0.82  val loss: 1.9955 val acc: 0.23 test loss: 0.2190 val acc: 0.22
Epoch: 11 train loss: 0.8370 train acc: 1.00  val loss: 1.7554 val acc: 0.38 test loss: 0.3870 val acc: 0.39
Epoch: 16 train loss: 0.4960 train acc: 1.00  val loss: 1.6560 val acc: 0.42 test loss: 0.4450 val acc: 0.45
Epoch: 21 train loss: 0.2625 train acc: 1.00  val loss: 1.5750 val acc: 0.43 test loss: 0.4490 val acc: 0.45
Epoch: 26 train loss: 0.1403 train acc: 1.00  val loss: 1.5674 val acc: 0.45 test loss: 0.4420 val acc: 0.44
Epoch: 31 train loss: 0.0737 train acc: 1.00  val loss: 1.6039 val acc: 0.45 test loss: 0.4540 val acc: 0.45
Epoch: 36 train loss: 0.0448 train acc: 1.00  val loss: 1.6474 val acc: 0.45 test loss: 0.4700 val acc: 0.47
Epoch: 41 train loss: 0.0269 train acc: 1.00  val loss: 1.6880 val acc: 0.46 test loss: 0.4710 val acc: 0.47
Epoch: 46 train loss: 0.0182 train acc: 1.00  val loss: 1.7079 val acc: 0.44 test loss: 0.4700 val acc: 0.47
Epoch: 51 train loss